In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.applications import ResNet50, EfficientNetB7, VGG16
from keras.preprocessing.image import ImageDataGenerator


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
#Extracting Data
! unzip /content/drive/MyDrive/species_resized_256.zip -d /content/images &> /dev/null

In [ ]:
#Extracting saved model
! unzip /content/drive/MyDrive/model.zip -d /content &> /dev/null

In [ ]:
# Set the main data directory
data_dir = '/content/images/'

# Get a list of all labels (subdirectories)
labels = os.listdir(data_dir)

# Create train, test, and validation directories
train_dir = '/content/train'
test_dir = '/content/test'
val_dir = '/content/validation'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Loop through each label and split data into train, test, and validation
for label in labels:
    label_dir = os.path.join(data_dir, label)
    all_files = os.listdir(label_dir)

    # Split the data into train and test
    train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

    # Split the test set into test and validation
    test_files, val_files = train_test_split(test_files, test_size=0.5, random_state=42)

    # Create label-specific directories in train, test, and validation
    train_label_dir = os.path.join(train_dir, label)
    test_label_dir = os.path.join(test_dir, label)
    val_label_dir = os.path.join(val_dir, label)

    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(test_label_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)

    # Move files to their respective directories
    for filename in train_files:
        shutil.move(os.path.join(label_dir, filename), os.path.join(train_label_dir, filename))

    for filename in test_files:
        shutil.move(os.path.join(label_dir, filename), os.path.join(test_label_dir, filename))

    for filename in val_files:
        shutil.move(os.path.join(label_dir, filename), os.path.join(val_label_dir, filename))


In [ ]:
# Set up ImageDataGenerator for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    cval=0,
    brightness_range=(0.7,1.3)
)

# Set up ImageDataGenerator for testing and validation data (only rescaling)
test_val_datagen = ImageDataGenerator(rescale=1./255)

# Batch size
batch_size = 64

# Target size of images
target_size = (224, 224)

# Generate training data from the directory
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

# Generate testing data from the directory
test_generator = test_val_datagen.flow_from_directory(
    '/content/test',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

# Generate validation data from the directory
validation_generator = test_val_datagen.flow_from_directory(
    '/content/validation',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

Found 28509 images belonging to 10 classes.
Found 3562 images belonging to 10 classes.
Found 3570 images belonging to 10 classes.


In [ ]:
def predict_generator(model,test_generator):

  # Get the number of batches in the test generator
  num_batches = len(test_generator)

  # Initialize lists to store true and predicted labels
  true_labels = []
  predicted_labels = []

  # Loop through each batch in the test generator
  for i in range(num_batches):
      # Get the batch of data
      batch_data, batch_true_labels = test_generator[i]

      # Predict the labels for the batch
      batch_predictions = model.predict(batch_data)

      # Get the predicted class indices for each sample in the batch
      batch_predicted_indices = np.argmax(batch_predictions, axis=1)

      # Extend the true and predicted label lists
      true_labels.extend(np.argmax(batch_true_labels, axis=1))
      predicted_labels.extend(batch_predicted_indices)

  # Calculate accuracy
  accuracy = accuracy_score(true_labels, predicted_labels)

  # Print results
  print(f"Accuracy: {accuracy * 100:.2f}%")
  print(classification_report(true_labels,predicted_labels))


In [ ]:
# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False

# Create a new model with custom dense layers
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit_generator(train_generator, epochs=10, validation_data=validation_generator, verbose=2)

# Make predictions on the test data and prints the results
predict_generator(model,test_generator)

In [ ]:
# Load the EfficientNetB7 model
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Create a new model with custom dense layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit_generator(train_generator, epochs=10, validation_data=validation_generator, verbose=2)

# Make predictions on the test data and prints the results
predict_generator(model,test_generator)

In [ ]:
# Load the EfficientNetB7 model
base_model = VGG16(weights="imagenet",include_top=False,input_shape=(224,224,3))

# Freeze the pre-trained layers
for layer in base_model.layers:
  layer.trainable = False

# Create a new model with custom dense layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])

# Create a callback that saves the model's weights
checkpoint_path = "model_checkpoint.h5"
checkpoint = ModelCheckpoint(checkpoint_path,monitor='val_accuracy',save_best_only=True,mode='max',verbose=1)

# Train the model with the callback
history = model.fit_generator(train_generator,epochs=10,validation_data=validation_generator,verbose=2,callbacks=[checkpoint])

# Unfreeze the pre-trained layers
for layer in model.layers:
  layer.trainable = True

# Compile the model again with a low learning rate to fine-tune the model
model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=['accuracy'])

# Train the model with the callback
history = model.fit_generator(train_generator, epochs=5, validation_data=validation_generator, verbose=2,callbacks=[checkpoint])

# Make predictions on the test data and print the results
predict_generator(model,test_generator)

In [ ]:
# Load the EfficientNetB7 model
base_model = VGG16(weights="imagenet",include_top=False,input_shape=(224,224,3))

# Create a new model with custom dense layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=['accuracy'])

# Specify the path to the checkpoint
checkpoint_path = "model_checkpoint.h5"

# Load the model weights from checkpoint_path
model.load_weights(checkpoint_path)

# Create a callback that saves the model's weights
checkpoint = ModelCheckpoint(checkpoint_path,monitor='val_accuracy',save_best_only=True,mode='max',verbose=1)

# Train the model with the callback
history = model.fit_generator(train_generator, epochs=5, validation_data=validation_generator, verbose=2,callbacks=[checkpoint])

# Make predictions on the test data and print the results
predict_generator(model,test_generator)